In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (10, 6)
import datetime as dt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

from plotly.offline import iplot
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly import tools

import os

if not os.path.exists("images"):
    os.mkdir("images")

In [ ]:
file = 'docs/metall_rates.csv'
df = pd.read_csv(file, decimal=',')
df.info()

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'],
                           format = '%d.%m.%Y')
df.info()

In [ ]:
df.date = df.date.sort_values(ascending=False)
df.head()

In [ ]:
# create date_range
# end - 2020-10-23
yeardays = pd.date_range(start=pd.datetime(2018, 1, 1),
                           end=pd.datetime(2020, 10, 23))
yeardays

In [ ]:
# convert to DataFrame
yeardays = pd.DataFrame(yeardays)
yeardays = yeardays.rename(columns={0: "date"})
yeardays.info()

In [ ]:
# merge two df's 
ndf = pd.merge(left=yeardays,
               right=df,
               on="date",
              how='outer')
ndf.head()

In [ ]:
ndf = ndf.ffill(axis = 0)
ndf.head(15)

In [ ]:
df = ndf
df.info()

In [ ]:
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month

df.columns

In [ ]:
df.head()

In [ ]:
# df['formatted_date'] = pd.to_datetime(df['datetime'])
# df['month'] = df['date'].apply(lambda x: x.month)
# df['week'] = df['date'].apply(lambda x: x.weekofyear)
# df['day'] = df['date'].apply(lambda x: x.days)

In [ ]:
# mean per week
df['mean_per_week_gold'] = df.groupby('week')['gold'].transform('mean')
df['mean_per_week_silver'] = df.groupby('week')['silver'].transform('mean')
df['mean_per_month_gold'] = df.groupby('month')['gold'].transform('mean')
df['mean_per_month_silver'] = df.groupby('month')['silver'].transform('mean')

In [ ]:
# rolling gold
df['rolling_7_gold'] = df['gold'].rolling(window=7).mean()
df['rolling_35_gold'] = df['gold'].rolling(window=35).mean()
df['rolling_50_gold'] = df['gold'].rolling(window=50).mean()

In [ ]:
# rolling silver
df['rolling_7_silver'] = df['silver'].rolling(window=7).mean()
df['rolling_35_silver'] = df['silver'].rolling(window=35).mean()
df['rolling_50_silver'] = df['silver'].rolling(window=50).mean()

In [ ]:
# exponential mean gold
df['ewm_7_gold'] = df['gold'].ewm(span=7, adjust=False).mean()
df['ewm_35_gold'] = df['gold'].ewm(span=35, adjust=False).mean()
df['ewm_50_gold'] = df['gold'].ewm(span=50, adjust=False).mean()

In [ ]:
# exponential mean silver
df['ewm_7_silver'] = df['silver'].ewm(span=7, adjust=False).mean()
df['ewm_35_silver'] = df['silver'].ewm(span=35, adjust=False).mean()
df['ewm_50_silver'] = df['silver'].ewm(span=50, adjust=False).mean()

In [ ]:
df.head(15)

In [ ]:
df.index = df['date']
df.drop('date',axis = 1, inplace = True)
df.head()

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
# silver_list = ['silver', 
# #                'mean_per_week_silver', 
# #                'rolling_7_silver',
# #                'rolling_35_silver',
#                'rolling_50_silver']
# gold_list = ['gold',
# #              'mean_per_week_gold', 
# #              'rolling_7_gold',
# #              'rolling_35_gold',
#              'rolling_50_gold']

In [ ]:
# !pip3 install psutil
# !pip3 install -U kaleido

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.silver, 
                         name='Цена на серебро'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.rolling_50_silver, 
                         name='Скользящая средняя за 50 дней'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.ewm_50_silver, 
                         name='Экспоненициальная скользящая средняя за 50 дней'))

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
#     gridcolor='LightPink',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на серебро за 2018 - 2020 год (цена дневная р/грамм и недельная средняя)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h"),
    barmode='overlay',
    margin=dict(l=50, r=50, t=30, b=30))



#     xaxis_title="Месяцы",
#     yaxis_title="Рубли",


fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'Месяцы')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'Рубли')


fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df.gold, 
    name='Цена на золото'))
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df.rolling_50_gold, 
    name='Скользящая средняя за 50 дней'))
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df.ewm_50_gold, 
    name='Экспоненициальная скользящая средняя за 50 дней'))

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на золото за 2018 - 2020 год (цена дневная р/грамм и недельная средняя)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h"),
    barmode='overlay',
    margin=dict(l=50, r=50, t=30, b=30))


fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'Месяцы')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'Рубли')


fig.show()

In [ ]:
def seasonal_decompose_plot(result, metall):
    trace1 = go.Scatter(
        x=result.observed.index,
        y=result.observed
    )

    trace2 = go.Scatter(
        x=result.trend.index,
        y=result.trend,
        xaxis='x2',
        yaxis='y2'
    )

    trace3 = go.Scatter(
        x=result.seasonal.index,
        y=result.seasonal,
        xaxis='x3',
        yaxis='y3'
    )

    trace4 = go.Scatter(
        x=result.resid.index,
        y=result.resid,
        xaxis='x4',
        yaxis='y4'
    )

    data = [trace1, trace2, trace3, trace4]
    layout = go.Layout(
        showlegend=False,
    )

    fig = make_subplots(
        rows=4,
        cols=1,
        print_grid=False
    )

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(trace3, 3, 1)
    fig.append_trace(trace4, 4, 1)

    fig['layout'].update(
        plot_bgcolor='rgba(0,0,0,0)',
        title=(f"Декомпозиция временного ряда 2020 ({metall})"),
        title_x = 0.5,
        showlegend=False,
        height=1024,
        yaxis=dict(title="График",
                   gridcolor='lightblue'),
        xaxis = dict(gridcolor='lightblue'),
        
        yaxis2=dict(title="Тренд",
                   gridcolor='lightblue'),
        xaxis2 = dict(gridcolor='lightblue'),
        
        yaxis3=dict(title="Сезонная компонента",
                   gridcolor='lightblue'),
        xaxis3 = dict(gridcolor='lightblue'),
        
        yaxis4=dict(title="Остаточная компонента",
                   gridcolor='lightblue'),
        xaxis4=dict(title="Временной ряд",
                    gridcolor='lightblue'))


    plotly.offline.iplot(fig)

In [ ]:
# ЭТОТ ГРАФИК

In [ ]:
start_01_01_2020 = pd.to_datetime("2020-01-01").date()

df2020 = df[start_01_01_2020:]
# df_last_month = df[last_month:]
df2020.tail()

In [ ]:
result = seasonal_decompose(df2020['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [ ]:
result = seasonal_decompose(df2020['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [ ]:
# тут построить разложения по месяцам

In [ ]:
last_month = pd.to_datetime("2020-10-01").date()

In [ ]:
df.tail()

In [ ]:
df_last_month = df[last_month:]
df_last_month.tail(35)

In [ ]:
result = seasonal_decompose(df_last_month['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [ ]:
result = seasonal_decompose(df_last_month['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [ ]:
# # золото и серебро

# # x = df['date']
# y1 = df['gold']
# y2 = df['silver']

# # Plot Line1 (Left Y Axis)
# fig, ax1 = plt.subplots(1,1,figsize=(16,9), dpi= 80)
# # ax1.plot(x, y1, color='tab:red')
# ax1.plot(y1, color='tab:red')

# # Plot Line2 (Right Y Axis)
# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# # ax2.plot(x, y2, color='tab:blue')
# ax2.plot(y2, color='tab:blue')

# # Decorations
# # ax1 (left Y axis)
# ax1.set_xlabel('Месяцы', fontsize=20)
# # ax1.tick_params(axis='x', rotation=0, labelsize=12)
# ax1.set_ylabel('Цены на золото', color='tab:red', fontsize=20)
# ax1.tick_params(axis='y', rotation=0, labelcolor='tab:red' )
# ax1.grid(alpha=.4)

# # ax2 (right Y axis)
# ax2.set_ylabel("Цены на серебро", color='tab:blue', fontsize=20)
# ax2.tick_params(axis='y', labelcolor='tab:blue')
# # ax2.set_xticks(np.arange(0, len(x), 60))
# # ax2.set_xticklabels(x[::60], rotation=90, fontdict={'fontsize':10})
# ax2.set_title("Сравнение динамики цен на золото и серебро", fontsize=22)
# fig.tight_layout()
# plt.show()

In [ ]:
# y1 = df['gold']
# y2 = df['silver']

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces

fig.add_trace(
    go.Scatter(x=df.index, 
               y=df.gold, 
               name="Золото",
               line=dict(color='#f5c541')
              ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.index, 
               y=df.silver, 
               name="Серебро",
               line=dict(color='#1167b1')
              ),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Сравнение динамики цен на золото и серебро (2018 - 2020 год)",
    title_x = 0.47,
    width=1024,
    height=600,
    legend=dict(x=.45, 
                xanchor="center", 
                orientation="h")
)

# Set x-axis title
fig.update_xaxes(title_text="<b>Месяцы</b>",
                 gridcolor='lightblue')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Золото</b> (р./грамм)", 
                 secondary_y=False,
#                  gridcolor='#f5c541'
                )
fig.update_yaxes(title_text="<b>Серебро</b> (р./грамм)", 
                 secondary_y=True,
#                  gridcolor='#1167b1'
                )

fig.show()

In [ ]:
# разбить внизу график на 3 года:
# https://community.plotly.com/t/multiple-x-axes-with-different-ticks-depending-on-zoom-level/38634

In [ ]:
sell_silver = pd.DataFrame({'y': ['2018-01-01', '2019-01-01', '2020-01-01'], 
                          'Тираж': [150000, 100000, 300000],
                          'Реализовано': [47902, 64425, 174338]})
sell_silver['y'] = pd.to_datetime(sell_silver['y'],
                                  format = '%Y-%m-%d')
sell_silver['y'] = sell_silver['y'].dt.year
sell_silver = sell_silver.set_index('y')
sell_silver.head()

In [ ]:
sell_gold = pd.DataFrame({'y': ['2018-01-01', '2019-01-01', '2020-01-01'], 
                          'Тираж': [150000, 300000, 500000],
                          'Реализовано': [93367, 210172, 424876]})
sell_gold['y'] = pd.to_datetime(sell_gold['y'],
                                  format = '%Y-%m-%d')
sell_gold['y'] = sell_gold['y'].dt.year
sell_gold = sell_gold.set_index('y')
sell_gold.head()

In [ ]:
# test
#  metall + sell

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 

data = [
# scatter-line
    go.Scatter(x=df.index, 
               y=df.silver, 
               name="Золото",
               line=dict(color='#f5c541')
              ),
    
    go.Bar(x=df.index, # assign x as the dataframe column 'x'
           y=sell_silver['Тираж'],
           name = 'Тираж'
          ),
    
    go.Bar(x=df.index, # assign x as the dataframe column 'x'
           y=sell_silver['Реализовано'],
           name = 'Реализовано'
          )
]

for i in data:
    fig.append_trace(i)


# Add figure title
fig.update_layout(
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Серебро (2018 - 2020 год)",
    title_x = 0.47,
    width=1024,
    height=600,
    legend=dict(x=.45, 
                xanchor="center", 
                orientation="h")
)

# Set x-axis title
fig.update_xaxes(title_text="<b>Годы</b>",
                 gridcolor='lightblue')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
                 secondary_y=False,
#                  gridcolor='#f5c541'
                )
fig.update_yaxes(title_text="<b>Шт.</b>", 
                 secondary_y=True,
                 gridcolor='#1167b1'
                )
fig.update_yaxes(title_text="<b>Шт.</b>", 
                 secondary_y=True,
                 gridcolor='#1167b1'
                )

fig.show()

In [ ]:
'''
ПГ


Паш, под этой ячейкой пример того, что я бы хотел изобразить:
линия и барплоты.

Идея в том, чтобы протянуть линию по стоимости металла за три года и наложить "изготовлено / продано"

как я понимаю, проблема в том, что металл у меня - цена на каждый день, а "изготовлено / продано" - суммарно по годам.

Посмотри, пожалуйта, код из примера ниже можно ли как-то переложить, чтобыон решил задачу

'''




In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])#this a one cell subplot

x = ['Shenzen', 'Australian Open', 'Fed Cup', 'Dubai', 'Indiana Wells', 'Miami', 
                  'Stuttgart', 'Madrid', 'Rome', 'French Open', 'Birmingham', 'Wimbledon', 
                  'Toronto', 'Cincinnati', 'US Open', 'Guangzhou', 'Wuhan', 'Beijing', 'WTA Finals'] 
y = [280, 430, 0, 900, 1000, 390, 185, 10, 350, 70, 100, 10, 585, 585, 780, 60, 105, 10, 370]
yy = [280, 710, 710, 1610, 2610, 3000, 3185, 3195, 3545, 3615, 3715, 3725, 4310, 4895, 5675,
                       5735, 5840, 5850, 6220]


trace1 = go.Bar(x=x,
                y=y,
                name='Points',            
                marker=dict(color= 'rgba(220,49,72, 0.8)', 
                            line= dict(width= 1)))
 

trace2 = go.Scatter(x=x, 
                    y=yy, 
                    marker= dict(line= dict(width= 1), 
                           size= 8), 
                    line=dict(color= '#B0122C', width= 1.5),
                    name= 'Accumulated pts')

#The first trace is referenced to the default xaxis, yaxis (ie. xaxis='x1', yaxis='y1')
fig.add_trace(trace1, secondary_y=False);

#The second trace is referenced to xaxis='x1'(i.e. 'x1' is common for the two traces) 
#and yaxis='y2' (the right side yaxis)

fig.add_trace(trace2, secondary_y=True);
fig.update_layout(autosize= False, 
                  width= 700, 
                  height= 500, 
                  legend= dict(x=1.15, 
                               y=1),
                  hovermode='x');
fig.update_xaxes(tickangle= -45, 
                 autorange=True,
                 tickfont=dict(size= 10), 
                 title= 'Tournaments', 
                 type= 'category');

fig.update_yaxes(range= [0, 1050], #left yaxis
                 title= 'Points', secondary_y=False);
fig.update_yaxes(range= [250, 6300],#right yaxis
                 showgrid= False, 
                 title= 'Cumulative sum of points',
                 secondary_y=True);
fig.show()
# from chart_studio.plotly import  iplot
# iplot(fig, filename='Example-Forum')

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])#this a one cell subplot

x = sell_gold.index
y = df.gold
yy = sell_gold['Тираж']


trace1 = go.Scatter(x=x, 
                    y=y, 
                    marker= dict(line= dict(width= 1), 
                           size= 8), 
                    line=dict(color= '#B0122C', width= 1.5),
                    name= 'Accumulated pts')


trace2 = go.Bar(x=x,
                y=yy,
                name='Золото',            
                marker=dict(color= 'rgba(220,49,72, 0.8)', 
                            line= dict(width= 1)))
 



#The first trace is referenced to the default xaxis, yaxis (ie. xaxis='x1', yaxis='y1')
fig.add_trace(trace1, secondary_y=False);

#The second trace is referenced to xaxis='x1'(i.e. 'x1' is common for the two traces) 
#and yaxis='y2' (the right side yaxis)

fig.add_trace(trace2, secondary_y=True);



fig.update_layout(autosize= False, 
                  width= 700, 
                  height= 500, 
                  legend= dict(x=1.15, 
                               y=1),
                  hovermode='x');
fig.update_xaxes(tickangle= -45, 
                 autorange=True,
                 tickfont=dict(size= 10), 
                 title= 'Tournaments', 
                 type= 'category');

fig.update_yaxes(
#     range= [0, 1050], #left yaxis
    title= 'Points', secondary_y=False);
fig.update_yaxes(
#     range= [250, 6300],#right yaxis
    showgrid= False, 
    title= 'Cumulative sum of points',
    secondary_y=True);
fig.show()
# from chart_studio.plotly import  iplot
# iplot(fig, filename='Example-Forum')

In [ ]:
#
# 1. перестроить на 2018-2020


In [ ]:
# сравнения по месяцам

In [ ]:
df.info()

In [ ]:
# months = list(set(list(df.month)))
# months
# df[df["month"]>=1]

In [ ]:
# def plot_month_plotly(df, metall):
#     months = list(set(list(df.month)))
#     data = []
#     df = df[df["month"]>=1]
#     for month in months:
#         monthdf = df[df["month"] == month]
#         trace = go.Scatter(
#             x = monthdf['month'],
#             y = monthdf[metall],
#             mode = 'lines',
#             name = month
#         )
#         colorscale='Jet',
#         data.append(trace)
# #    iplot(data, filename='line-mode')
    
    
#         layout = go.Layout(
#         title = 'Сравнение месяц к месяцу (2020)'
#     )

#     fig = go.Figure(data=data, layout=layout)
#     iplot(fig, filename='line-mode')

In [ ]:
df2020.columns

In [ ]:
def plot_month_plotly(df, metall):
    pv_test = pd.pivot_table(df,
                        index=["day"],
                        columns=["month"],
                        values=[metall])

    pv_test.columns = [col[1] for col in pv_test.columns]

    data = [go.Scatter(x = pv_test.index,
                       y = pv_test[col],
                       name = col)
            for col in pv_test.columns]

    iplot(data)

In [ ]:
plot_month_plotly(df2020, 'gold')

In [ ]:
plot_month_plotly(df2020, 'silver')

In [ ]:
invest_3_years = pd.DataFrame({'y': [2018, 2019, 2020], 
                               'Золото': [150000, 300000, 500000],
                              'Серебро': [150000, 100000, 300000]})
invest_3_years = invest_3_years.set_index('y')
invest_3_years

In [ ]:
invest_3_years.index.unique()

In [ ]:
for i in invest_3_years.index.unique():
    print(invest_3_years[invest_3_years.index == i])

In [ ]:
list(invest_3_years.columns)

In [ ]:
for i in invest_3_years.index.unique():
    labels = list(invest_3_years.columns)
    values = invest_3_years[invest_3_years.index == i].values[0]
    print(labels, values)

In [ ]:
metall_colors = ['#f5c541','#1167b1']

for i in invest_3_years.index.unique():
    labels = list(invest_3_years.columns)
    values = invest_3_years[invest_3_years.index == i].values[0]
    fig = go.Figure([go.Pie(labels=labels, 
                            values=values, 
                            hole=.3,
                            marker_colors=metall_colors,
                            textinfo='label+percent')])
    fig.update_layout(title_text='Соотношение золотых / серебряных инвестиционных монет в шт., '+str(i)+' год',
                      title_x = 0.5,
                      showlegend=False)
    fig.show()

In [ ]:





# for i in invest_3_years.index.unique():
#     go.Pie(labels=invest_3_years.columns, values=invest_3_years[invest_3_years.index == i])
#     fig = go.Figure(
#         go.Pie(labels=invest_3_years.columns, values=invest_3_years[invest_3_years.index == i])
#     )
#     fig.show()

In [ ]:
# график сезонов
# def metall_trend_monthly(metall, tit):
#     mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:brown', 'tab:grey', 'tab:pink', 'tab:olive', 'deeppink', 'steelblue']
#     months = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь']
#     plt.figure(figsize=(16,10), dpi= 80)
#     plt.suptitle('Сравнение месячных трендов за 2020 год', fontsize=15)
#     plt.title(str(tit))
#     plt.xlabel('День месяца', fontsize=14)
#     plt.ylabel('Цена', fontsize=14)
#     lines = []
#     for i, y in enumerate(df[metall].groupby(pd.Grouper(freq='M'))):
#         p,=plt.plot(range(len(y[1])),y[1], color=mycolors[i], label=months[i])
#         lines.append(p)
#     plt.legend(handles=lines)

In [ ]:
# metall_trend_monthly('silver', 'Серебро')

In [ ]:
# metall_trend_monthly('gold', 'Золото')

In [ ]:
# from datetime import date

In [ ]:
# allbanks = pd.read_csv('docs/allbanks.csv', 
#                        encoding='cp1251', 
#                        sep=';', 
#                        index_col=0)
# allcoins = pd.read_csv('docs/allcoins.csv', 
#                        encoding='cp1251', 
#                        sep=';', 
#                        index_col=0)
# prices = pd.read_csv('docs/prices.csv', 
#                      encoding='cp1251', 
#                      sep=';')

In [ ]:
# allbanks.head()

In [ ]:
# allcoins.head()

In [ ]:
# allcoins.DT = pd.to_datetime(allcoins.DT, unit='s').dt.date
# allcoins.head()

In [ ]:
# prices.head()

In [ ]:
# prices.date = pd.to_datetime(prices.date, unit='s').dt.date
# prices.head()

In [ ]:
# result_df = prices.merge(allcoins, left_on = 'id_coin', right_index=True)
# result_df.head()

In [ ]:
# result_df = result_df.merge(allbanks, left_on = 'id_bank', right_index=True)
# result_df.head()

In [ ]:
# visualization

In [ ]:
# allcoins['metal'].unique()

In [ ]:
# nedrag_list = ['Медно-никелевый сплав',
#                'Сталь с латунным/никелевым гальваническим покрытием',
#                'Сталь с латунным гальваническим покрытием', 
#                'Латунь/Мельхиор',
#                'Сталь с никелевым гальваническим покрытием', 
#                'Мельхиор',
#                'Нейзильбер', 
#                'Медь, никель', 
#                'Латунь',
#                'Латунь/Медь, никель', 
#                'Нейзильбер/Латунь',
#                'Медь, цинк/Медь, никель']

In [ ]:
# gold_list = ['Au-999',
#              'Au-900']

In [ ]:
# silver_list = ['Ag-925', 
#                'Ag-999', 
#                'Ag-900', 
#                'Ag-500']

In [ ]:
# combine_metal_list = ['Ag-900/Au-900', 
#                       'Ag-925/Au-999', 
#                       'Ag-925/позолота', 
#                       'Pd-999', 
#                       'Pt-999', 
#                       'AU-900/Ag-925', 
#                       'Ag-925/Au-999', 
#                       'Ag-900/Au-900', 
#                       'Au-900/Ag-900']

In [ ]:
# allcoins_gold = allcoins[allcoins['metal'].isin(gold_list)]
# allcoins_gold.metal.unique()

In [ ]:
# allcoins_silver = allcoins[allcoins['metal'].isin(silver_list)]
# allcoins_silver.metal.unique()

In [ ]:
# allcoins_nedrag = allcoins[allcoins['metal'].isin(nedrag_list)]
# allcoins_nedrag.metal.unique()

In [ ]:
# allcoins_combine = allcoins[allcoins['metal'].isin(combine_metal_list)]
# allcoins_combine.metal.unique()

In [ ]:
# startdate = pd.to_datetime("2017-01-01").date()

In [ ]:
# allcoins_gold_3_years = allcoins_gold[allcoins_gold['DT'] >= startdate]
# allcoins_gold_3_years.head()

In [ ]:
# allcoins_silver_3_years = allcoins_silver[allcoins_silver['DT'] >= startdate]
# allcoins_silver_3_years.head()

In [ ]:
# allcoins_nedrag_3_years = allcoins_nedrag[allcoins_nedrag['DT'] >= startdate]
# allcoins_nedrag_3_years.head()

In [ ]:
# allcoins_combine_3_years = allcoins_combine[allcoins_combine['DT'] >= startdate]
# allcoins_combine_3_years.head()

In [ ]:
# def get_plot_count(datafr, i):
#     plt.figure(figsize=(10,5))
#     ax = sns.countplot(datafr['Nominal'])
#     ax.set(xlabel='Номинал монет', 
#            ylabel='Количество видов')
#     plt.suptitle('Количество видов монет за последние 3 года', size = 14)
# #     plt.figsize=(20, 20)
#     plt.title(str(i), size = 12)

#     plt.show()

In [ ]:
# get_plot_count(allcoins_gold_3_years, 'золото')

In [ ]:
# get_plot_count(allcoins_silver_3_years, '(серебро)')

In [ ]:
# get_plot_count(allcoins_nedrag_3_years, '(недрагоценные металлы)')

In [ ]:
# ВОТ ОТСЮДА

In [ ]:
# result_df.head()

In [ ]:
# result_df_stat = result_df[['sell_price','Nominal']]
# result_df_stat.groupby('Nominal').describe()

In [ ]:
# .groupby('Nominal').describe()

In [ ]:
# result_df_gold = result_df[result_df['metal'].isin(gold_list)]
# result_df_gold.Nominal.unique()

In [ ]:
# result_df_silver = result_df[result_df['metal'].isin(silver_list)]

In [ ]:
# result_df_nedrag = result_df[result_df['metal'].isin(nedrag_list)]

In [ ]:
# describe

In [ ]:
# result_df_gold[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
# result_df_silver[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
# result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
# result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [ ]:
# result_df_nedrag[['sell_price','coin_name', 'rus_name']].groupby('sell_price').describe()

In [ ]:
# result_df_gold.info()

In [ ]:
# result_df_gold.Nominal.unique()

In [ ]:
# for i in set(result_df_gold.Nominal.unique()):
#     hidden = result_df_gold
#     hidden['Nominal'] = result_df_gold[result_df_gold['Nominal']==str(i)]
#     hidden.groupby('cat_number').describe()

In [ ]:
# result_df_gold[['sell_price','Nominal']].head()

In [ ]:
# res_clear = result_df_gold[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>30000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [ ]:
# nominal_list

In [ ]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="gold",
#                 bins = 10).set(title=i)
#     plt.show()

In [ ]:
# res_clear = result_df_silver[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>2000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [ ]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="silver",
#                 bins=100).set(title=i)
#     plt.show()

In [ ]:
# df.to_csv('docs/drags.csv')